In [1]:
with open(r'../Netcrawler/fitbit_extracted.txt', 'r', encoding='utf-8') as file:
    text_data = file.read()

In [2]:
text_data=text_data.strip('[]').split(', ')
text_data

# Lowercase all the paragraphs in the list
text_data = [paragraph.lower() for paragraph in text_data]

In [3]:
text_data

["\n1.\xa0 green star means you met your goal for amount of sleep.\xa02. as long as the line is green and below the 'high' line",
 'that means your eov is low (or at least not high)',
 "which is good.\xa0 when line goes above 'high'",
 'graph turns yellow',
 "at least on android - can't guarantee graph looks same on ios.\xa03. to insert picture/screen cap",
 'click camera icon above input box',
 "then 'choose files' lets you select the location on your computer where file is located.\n\nbefore posting",
 're-read to see if it would make sense to someone else not looking at your fitbit or phone.\nusing an inspire 2 tracker. \xa0consistently',
 'every night',
 'i get a spike in my heart rate that falls mostly in the fat burn range but sometime goes into the next level. \xa0the duration is variable',
 'sometimes only one reading',
 'sometimes spanning as much as 15 min. \xa0it always occurs when the tracker indicates i am in light sleep but a few times during deep sleep (when the duration

In [4]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Step 1: Text Cleaning
def clean_text(text):
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)#

    # Remove newline
    text = re.sub(r'\\n', '', text)
    
    # Remove special characters except spaces
    text = re.sub(r'[^\w\s]', '', text)

    # Remove all numbers
    text = re.sub(r'\d', '', text)
    
    non_latin_pattern = re.compile(r'[^\x00-\x7F]+')

    text = re.sub(non_latin_pattern, ' ', text)

    # Remove extra spaces and trim the text
    text = re.sub(r'\s+', ' ', text).strip()

    return text


cleaned_data = [clean_text(text) for text in text_data]
# delete xa0
cleaned_data = [text.replace('xa0', ' ') for text in cleaned_data]


In [5]:
# Step 2: Tokenization
nltk.download('punkt')  # Download the NLTK tokenizer data if not already downloaded

def tokenize_text(text):
    # Tokenize the text into words
    words = word_tokenize(text)
    return words

tokenized_data = [tokenize_text(text) for text in cleaned_data]


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dc78udot\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
import pandas as pd

# Specify the file path
csv_file_path = 'uscities.csv'
# Quelle: https://simplemaps.com/data/us-cities

# Read the CSV file into a pandas DataFrame
city = pd.read_csv(csv_file_path)

In [7]:
city_list = city['city'].tolist()
state_list = list(city['state_name'].unique())

In [8]:
# Step 3: Stopword Removal
# open stopword file (source: https://countwordsfree.com/stopwords)
with open(r'stop_words_english.txt', 'r', encoding='utf-8') as file:
    stopwordlist = file.read()

# Split the string into a list of stop words using line breaks as the separator
stopwordslist = stopwordlist.split('\n')

# Remove any empty strings from the list
stopwordslist = [word for word in stopwordlist if word.strip()]


otherwords = ["night", "time", "minute", "ive", "one", "got", "dont", "lot" 
                , "getting", "good", "much", "first", "give", "thank", "might"
                , "really", "im", "fitbit", "te", "would", "else", "cant" 
                , "sleep", "want", "back", "around", "month", "usaversa", "luxe", "aria"
                , "fit", "bit", "maryland", "flwant", "answered", "answer", "community", "moderator"
                , "original", "post", "sleeprepeatdave", "hour", "hours", "minute", "asleep"
                , "january", "february", "march", "april", "may", "june", "july", "august"
                , "september", "november", "december", "comparte", "tus"
                , "week", "month", "year", "sugerencias", "para", "visit", "dispositivos", "nuevos"
                , "edit", "subject", "weeks", "months", "minutes" ,"xperia", "android", "sony", "charge"
                , "sense", "dominique", "finlandionic", "australiaionic", "colinvictoria", "ipad", "iphone", "moto", "motorola"
                ,"doesnt", "dont", "doesn\'t", "don\'t", "day", "days", "didnt", "didn\'t", "io"
                , "karolien", "netherlands", "posting", "reread", "italian", "fitbitti", "davide", "invito", "partecipare"
                , "isnt", "ᕙᕗ", "havent", "nelle", "nostre", "discussioni"]


stopwordlistpers = set()
for item in city_list + state_list + otherwords:
    stopwordlistpers.add(item.lower())

stopwordlistpers = list(stopwordlistpers)

def remove_stopwords(tokens, stop_words):
    return [word for word in tokens if word not in stop_words]

# Remove common stopwords
filtered_data = [remove_stopwords(tokens, stopwordlist) for tokens in tokenized_data]

# Remove personalized stopwords
filtered_data = [remove_stopwords(tokens, stopwordlistpers) for tokens in filtered_data]

# Remove blanco
filtered_data = [sublist for sublist in filtered_data if sublist]


In [9]:
import nltk
#nltk.download('wordnet')
#nltk.download('omw-1.4')
# Step 4: Lemmatization
def lemmatize_tokens(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return lemmatized_tokens

lemmatized_data = [lemmatize_tokens(tokens) for tokens in filtered_data]


In [10]:
# Step 5: Rejoin Tokens into Text
preprocessed_data = [' '.join(tokens) for tokens in lemmatized_data]

In [11]:
%store preprocessed_data
%store lemmatized_data
%store cleaned_data

Stored 'preprocessed_data' (list)
Stored 'lemmatized_data' (list)
Stored 'cleaned_data' (list)
